In [12]:
import cv2
import mediapipe as mp
import numpy as np
import os
import csv
from matplotlib import pyplot as plt
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
mp_drawing_styles = mp.solutions.drawing_styles

In [15]:
landmarks = ['class']
for val in range(1,33+1):
    landmarks += ['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [16]:
with open('StanceCoords.csv',mode='w',newline='') as f:
    csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [17]:
def export_landmarks(results,action):
    try:
        keypoints = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0,action)

        with open('StanceCoords.csv',mode='a',newline='') as f:
            csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

In [19]:
# For webcam input:
cap = cv2.VideoCapture('stance.avi')
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.


    k=cv2.waitKey(1)
    if k == 97:
      export_landmarks(results,'narrow')
    if k == 115:
      export_landmarks(results,'normal')
    if k == 100:
      export_landmarks(results,'wide')
    cv2.imshow('MediaPipe Holistic', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [21]:
df = pd.read_csv('StanceCoords.csv')

In [22]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,wide,0.570135,0.134369,-0.438831,0.999988,0.575561,0.121968,-0.413995,0.999962,0.579743,...,0.227842,0.893901,0.677379,0.890459,0.176866,0.992561,0.459869,0.884188,0.037493,0.993154
1,wide,0.569072,0.134225,-0.430204,0.999987,0.575063,0.121948,-0.405360,0.999962,0.578914,...,0.217126,0.898623,0.677889,0.891058,0.168465,0.992565,0.459528,0.884570,0.027580,0.993517
2,wide,0.569075,0.134306,-0.431259,0.999987,0.575055,0.122014,-0.406499,0.999961,0.578892,...,0.217307,0.899698,0.677907,0.891080,0.168315,0.992589,0.459496,0.884774,0.027804,0.993588
3,wide,0.568969,0.134809,-0.453889,0.999987,0.574938,0.122279,-0.429685,0.999961,0.578693,...,0.221440,0.900595,0.677911,0.890992,0.181492,0.992654,0.459463,0.884775,0.030951,0.993648
4,wide,0.568990,0.135181,-0.453527,0.999987,0.574926,0.122465,-0.429175,0.999961,0.578661,...,0.213447,0.902571,0.678040,0.890893,0.210729,0.992734,0.459359,0.884803,0.022280,0.993779


In [23]:
X = df.drop('class',axis=1) #features
y = df['class'] #target

In [24]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1234)

In [25]:
y_test

1848    narrow
3271    narrow
2936    normal
2258      wide
3182    narrow
         ...  
1001    normal
2111    narrow
2645      wide
2095    narrow
3434    narrow
Name: class, Length: 1032, dtype: object

### Train Machine Learning Classification Model

In [26]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [27]:
pipelines = {
    'lr':make_pipeline(StandardScaler(),LogisticRegression()),
    'rc':make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [28]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

c:\Users\T470p\anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [29]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [30]:
fit_models['rc'].predict(X_test)

array(['narrow', 'narrow', 'normal', ..., 'wide', 'narrow', 'narrow'],
      dtype='<U6')

### Evaluate and Serialize Model

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score # Accuracy Metrics
import pickle

In [32]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,accuracy_score(y_test.values,yhat),
    precision_score(y_test.values,yhat,average='weighted'),
    recall_score(y_test.values,yhat,average='weighted'))

lr 1.0 1.0 1.0
rc 0.999031007751938 0.9990334062476014 0.999031007751938
rf 1.0 1.0 1.0
gb 1.0 1.0 1.0


In [33]:
yhat = fit_models['rf'].predict(X_test)

In [34]:
yhat[:10]

array(['narrow', 'narrow', 'normal', 'wide', 'narrow', 'narrow', 'wide',
       'wide', 'narrow', 'wide'], dtype=object)

In [35]:
y_test[:10]

1848    narrow
3271    narrow
2936    normal
2258      wide
3182    narrow
1859    narrow
118       wide
1184      wide
3260    narrow
454       wide
Name: class, dtype: object

In [36]:
with open('stance.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)


### Make Detections with the Model

In [37]:
with open('stance.pkl','rb') as f:
    model = pickle.load(f)

In [39]:
# For webcam input:
counter = 0
current_stage = ''
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    
    try:
        row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        X = pd.DataFrame([row],columns=landmarks[1:])
        body_lean_class = model.predict(X)[0]
        body_lean_prob = model.predict_proba(X)[0]
        print(body_lean_class,body_lean_prob)

        if body_lean_class == 'neutral' and body_lean_prob[body_lean_prob.argmax()] >= 0.7:
            current_stage = 'neutral'
            # print("huuuuuu")
        elif body_lean_class == 'left' and body_lean_prob[body_lean_prob.argmax()] >= 0.7:
            current_stage = 'left'
        elif body_lean_class == 'right' and body_lean_prob[body_lean_prob.argmax()] >= 0.7:
            current_stage = 'right'
        
        # Get status box
        cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1)

        #Display class
        cv2.putText(image,'CLASS'
            ,(95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,body_lean_class.split(' ')[0]
        , (90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

        #Display Probability
        cv2.putText(image,'PROB',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,str(round(body_lean_prob[np.argmax(body_lean_prob)],2))
        ,(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

        # #Display Probability
        # cv2.putText(image,'COUNT',(180,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        # cv2.putText(image,str(counter),(175,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

    except Exception as e:
        print(e)
        pass

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Holistic',image ) #cv2.flip(image, 1)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute